In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [5]:
!ls

adc.json  datalab  drive  sample_data


In [1]:
import keras
import datetime
from keras.models import Model, load_model
import os
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from keras.layers import Input, UpSampling2D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, Conv2DTranspose
import numpy as np
from skimage import io, color
from keras.preprocessing import image
import numpy as np
from skimage import io, color, transform
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist
from matplotlib import pyplot as plt
import os
image_size = 256

now = datetime.datetime.now()

image_path = "drive/app/input/"

def read_image(img_id, dir):
    try:
        img = load_img(dir + "/" + img_id, target_size=(256,256))
        img = img_to_array(img)
        return img
    except:
        return None

def convLayer(input, filters, kernel_size,  stride=1):
    return Conv2D(filters, kernel_size, padding="same", activation="relu", strides=stride)(input)

input_shape = (256, 256, 1)

model_input = Input(shape = input_shape)

model_output = convLayer(model_input, 32, (3, 3))
model_output = convLayer(model_output, 32, (3, 3), stride=2)

model_output = convLayer(model_output, 64, (3, 3))
model_output = convLayer(model_output, 64, (3, 3), stride=2)

model_output = convLayer(model_output, 128, (3, 3))
model_output = convLayer(model_output, 128, (3, 3), stride=2)

model_output = convLayer(model_output, 256, (3, 3))
model_output = convLayer(model_output, 256, (3, 3))

model_output = convLayer(model_output, 256, (3, 3))
model_output = convLayer(model_output, 256, (3, 3))

model_output = UpSampling2D((2, 2))(model_output)
model_output = convLayer(model_output, 128, (3, 3))

model_output = UpSampling2D((2, 2))(model_output)
model_output = convLayer(model_output, 64, (3, 3))

model_output = UpSampling2D((2, 2))(model_output)

model_output = Conv2D(2, (3, 3), activation="tanh", padding="same")(model_output)

model = Model(inputs=model_input, outputs=model_output)

model.load_weights("drive/app/output/my_model_weights2018-08-10 20:36.h5")
model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])



Using TensorFlow backend.


OSError: Unable to open file (unable to open file: name = 'drive/app/output/my_model_weights2018-08-10 20:36.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [8]:
# files = os.listdir(image_path)[512:522]
# Xtest = []
# for image in files:
#     img = read_image(image, image_path)
#     if not img is None:
#         img = np.array(img, dtype=float)
#         Xtest.append(img)

img = read_image("dani.jpg", image_path)
Xtest = []
Xtest.append(img)
Xtest = np.array(Xtest, dtype=float)
Xtest.reshape(Xtest.shape+(1,))
Xtest = 1.0/255*Xtest
inputs = color.rgb2lab(Xtest)[:, :, :, 0]
inputs = inputs.reshape(inputs.shape+(1,))

# Test model
output = model.predict(inputs)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = inputs[i][:,:,0]
    cur[:,:,1:] = buckets[np.argmax(output[i], axis=2)]
    io.imshow(color.lab2rgb(reverted))
    plt.show()

/usr/local/lib/python2.7/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
